# THINGSvision

## Settings

### Install thingsvision and dependencies

In [2]:
!pip install --upgrade thingsvision

In [3]:
!pip install ipywidgets

In [9]:
import torch
import tensorflow
import os
import thingsvision.vision as vision
import numpy as np

from thingsvision.model_class import Model
from google.colab import drive
from typing import Any

### Image and feature directories

Specify both `path/to/images` (input directory) and `path/to/features` (output directory) on your Google Drive. 
The image directory is expected to contain images that are saved similarly to `/dog/img_1.png` or `/cat/img_1.jpg`. 

In [10]:
image_dir = 'path/to/images'  # path/to/images in GDrive
output_dir = 'path/to/features' # path/to/output  in GDrive

Mount Google Drive 

In [12]:
mounted_dir = '/thingsvision'
drive.mount(mounted_dir, force_remount=True)

In [ ]:
full_image_path = os.path.join(mounted_dir, 'MyDrive', images_dir)
full_output_path = os.path.join(mounted_dir, 'MyDrive', output_dir)

#### Helper function to extract features

In [13]:
def extract_features(
                    model: Any,
                    module_name: str,
                    image_path: str,
                    out_path: str,
                    batch_size: int,
                    flatten_activations: bool,
                    apply_center_crop: bool,
                    clip: bool=False,
) -> np.ndarray:
    dl = vision.load_dl(
                        root=image_path,
                        out_path=out_path,
                        batch_size=batch_size,
                        transforms=model.get_transformations(apply_center_crop=apply_center_crop),
                        backend=backend,
    )
    features, _ = model.extract_features(
                                        data_loader=dl,
                                        module_name=module_name,
                                        batch_size=batch_size,
                                        flatten_acts=flatten_activations,
                                        clip=clip,
                                        return_probabilities=False,
    )
    return features

### Variables

In [14]:
backend = 'tf' # backend 'pt' for PyTorch or 'tf' for Tensorflow 
pretrained = True # use pretrained model weights
model_path = None # if pretrained = False (i.e., randomly initialized weights) set path to model weights
batch_size = 32 # use a power of two (this can be any size, depending on the number of images for which you aim to extract features)
apply_center_crop = True # center crop images (set to False, if you don't want to center-crop images)
flatten_activations = True # whether or not features (e.g., of Conv layers) should be flattened
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Select `model` and `layer` for which you want to extract image features. If you want to extract features from a `torchvision` model, use the model naming defined [here](https://pytorch.org/vision/stable/models.html) (e.g., `vgg16` if you want to use VGG-16). If you are uncertain about the naming and enumeration of the layers, use `model.show()` to see how specific layers called.

### VGG-16 (pretrained on ImageNet)

In [7]:
## load model
model_name = 'VGG16' 
model = Model(
            model_name,
            pretrained=pretrained,
            model_path=model_path,
            device=device,
            backend=backend,
)

2022-01-26 14:39:45.363506: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
## select layer

# NOTE: uncomment the line below, if you are uncertain about layer naming
#module_name = model.show() 
module_name = 'features.23' 

In [15]:
# extract features
features = extract_features(
                            model=model,
                            module_name=module_name,
                            image_path=full_image_path,
                            out_path=full_out_path,
                            batch_size=batch_size,
                            flatten_activations=flatten_activations,
                            apply_center_crop=apply_center_crop,
                            clip=False,
)

# save features to disk
vision.save_features(features, f'{full_output_dir}/features_{model_name}_{module_name}', 'npy')

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1